In [ ]:
import pickle
from astropy.io import fits
from ctapipe.io import EventSource
from ctapipe.visualization import CameraDisplay
import matplotlib.pyplot as plt
import numpy as np
import copy
from ctapipe.image import number_of_islands
from ctapipe.image import ImageProcessor
from ctapipe.image.muon import MuonProcessor
from ctapipe.calib import CameraCalibrator
import json
from traitlets.config import Config
from lstchain.io import replace_config, standard_config
from ctapipe_io_lst import calibration
from ctapipe.io import HDF5TableReader, read_table
from ctapipe.containers import FlatFieldContainer, MonitoringCameraContainer, MonitoringContainer, PedestalContainer, PixelStatusContainer, WaveformCalibrationContainer
import tables


from lstchain.image.muon import analyze_muon_event, tag_pix_thr

all_muons = [54110, 55656, 55942, 57623, 58136, 58798, 59194, 59570, 60407, 60422, 62395, 63033, 63412, 64971, 65272, 66251, 66737, 67143, 67175, 67772, 68483, 68495, 68755, 68902, 69300, 69891, 70855, 72398, 73259, 73261, 73725, 76581, 76871, 77277, 80267, 80982, 81229, 81606, 82140, 83970, 84318, 85580, 87424, 87427, 87921, 88388, 89632, 91351, 93414, 93971, 94651, 95006, 96661, 97072, 97242, 97335, 97713, 97785, 97891, 99847, 100267, 101084, 101156, 101606, 103028, 103475, 105020, 105713, 53917, 54661, 54702, 54740, 55022, 55439, 55550, 56455, 56800, 57193, 58178, 58184, 58254, 58424, 59011, 59264, 59777, 60099, 60736, 61877, 63390, 63725, 63441, 63725, 63962, 64004, 64745, 64748, 65331, 65482, 65720, 65765, 66084, 66147, 66230, 66260, 66388, 67310, 68018, 68134, 68266, 68874, 68894, 69037, 69199, 69308, 70990, 71065, 71861, 720021, 73068, 73286, 74956, 75009, 75306, 75968, 76378, 77121, 77910, 79039, 79152, 79979, 80233, 80306, 80416, 80600, 81641, 81788, 83896, 84018, 84065, 84680, 84731, 85242, 85492, 85731, 85873, 86765, 87951, 88893, 89081, 89124, 89192, 89278, 89987, 91143, 91653, 91745, 91965, 92114, 92579, 92644, 94793, 94948, 94968, 98779, 99404, 99653, 99657, 100128, 100569, 101446, 102579, 102937, 103709, 104049, 104534, 104880, 105110, 105408, 105978]
all_protons = [54598, 54899, 61665, 63822, 68675, 69169, 70354, 72471, 72806, 72838, 73946, 77882, 80717, 80824, 84099, 87150, 88823, 91043,  91909, 93262, 95463, 95941, 98786, 98892, 99798, 100446, 101227, 103180, 103365, 103916]
strict_protons = [54899, 61665, 63822, 68675, 70354, 72471, 72838, 73946, 77882, 80717, 80824, 87150, 91043,  91909, 93262, 95941, 98786, 98892, 99798, 100446, 103180, 103365, 103916]

In [ ]:
ring_9222 = [13, 14, 15, 15, 15, 15, 15, 16,  3, 39, 15, 14, 14,  0, 12, 13, 36,
       15, 14, 13, 14, 13, 13, 14, 12,  2, 14, 14, 15, 15, 15, 34, 14, 15,
       13, 14, 26,  9,  5, 15, 15, 14, 14, 14, 13, 14, 39, 15, 15, 14, 13,
       16, 14, 14, 32, 14, 15, 14, 15, 14, 14,  0, 15, 14, 16]

ring_2223 = [ 0, 14,  7, 15, 15, 14, 23, 17, 10, 10, 14, 13, 14, 14, 16, 13, 14,
       17, 14, 14, 16, 10, 22, 14, 38,  0, 14, 26,  0, 15,  3, 13, 12, 19,
       14, 14, 14,  3, 11, 14, 14, 15, 16, 13, 19,  7, 27, 11, 16, 14, 18,
       15,  0,  9, 15, 14, 32,  6, 28, 15, 17, 32, 13, 10,  6, 25, 15, 37,
       13, 26, 16, 26, 15, 30, 12,  1, 14, 13, 31, 27, 19, 15, 15, 20, 19,
       21, 33, 16, 15,  5, 36, 12, 38, 22, 18, 15, 12, 14, 14, 12,  4,  7,
       18, 25, 38, 26, 14, 14, 21, 16, 14, 15, 32, 31,  2,  0, 14, 14,  1,
        1, 19, 15,  3, 13, 22, 31, 25, 34, 17, 16, 15, 16,  9, 15, 34, 12,
        4, 13, 18, 32,  3, 15, 14, 31, 13, 17, 28,  9, 13, 39, 22, 15, 16,
       14, 15, 31,  0, 18,  3, 14, 19, 15, 16,  7, 15, 25, 12, 26, 15,  1,
       14, 28, 16, 15, 14, 13,  8, 11,  9,  2, 10, 13, 16]

proton_2223 = [19, 19, 18, 17, 17, 18, 17, 17, 17, 17, 17, 17, 18, 16, 17, 16, 17,
       18, 16, 18, 16, 17, 18, 17, 17, 18, 17, 16, 17, 16, 17, 17, 17, 17,
       16, 18, 15, 16, 16, 18, 17, 16]
proton_9222 = [20, 19, 20, 20, 22, 22, 22, 20, 19, 19, 21, 22, 20, 19, 20, 34, 19,
       19, 19, 17, 19, 19, 20, 18, 20, 19, 19, 20, 19, 18, 19, 19, 18, 21,
       19, 18, 19, 20, 17, 19, 19, 21]

proton_2438 = [16, 17, 16, 17, 16, 16, 17, 17, 17, 17, 18, 17, 19, 18, 18, 17, 17,
       20, 18, 18, 20, 19, 18, 16, 18, 17, 19, 20, 21, 18, 20, 19, 19, 18,
       19, 20, 18, 20, 19, 20, 18, 17, 18, 19, 20, 19, 17, 17, 18, 17, 18,
       16, 16, 17, 17, 18]

ring_2438 = [15, 15, 14, 15, 15, 15, 15, 14, 14, 13, 14, 13, 14, 14, 13, 13, 14,
       13, 15, 16, 14, 14, 15, 13, 14, 15, 14, 14, 15, 15, 13, 15, 15, 13,
       14, 14, 18, 14, 13, 14, 15, 14, 13, 14, 39, 13, 17, 14, 13, 14, 14,
       15, 14, 14, 12, 15, 13, 14, 14, 14, 13, 15, 13, 14, 15, 12, 15, 14,
       23, 14]

proton_25051 = [18, 17, 17, 17, 19, 20, 19, 20, 21, 20, 18, 19, 19, 21, 21, 21, 19,
       19, 20, 20, 20, 20, 19, 20, 19, 19, 20, 18, 19, 20, 19, 18, 18, 19,
       19, 19, 19]
ring_25051 = [14, 14, 15, 13, 14, 14, 12, 13, 14, 13, 13, 14, 15, 14, 15, 14, 14,
       13, 14, 14, 14, 14, 13, 15, 14, 15, 14, 14, 14, 14, 13, 15, 13, 13,
       14, 12, 14, 16, 14, 13, 16, 13, 14, 15, 13, 15, 13, 15, 13, 15, 15,
       14, 14, 13, 14, 14, 14, 13, 22, 15, 13, 13, 16, 15, 14,  7, 30, 14,
       13, 13, 11, 13, 14, 15, 15, 13, 15, 14, 13, 14, 13, 14, 13, 15]

# event not separated in time
# Separated by double cut (shower on sample 20 intensity > 0.9, ring on sample 12 intensity > 1.2)
# then pixels of shower was subtracted from ring pixels
#
ring_14406 = [13, 13, 13, 16, 20, 13, 12, 14, 15, 15, 14, 13, 13, 14, 12, 15, 14,
       14, 15, 13, 14, 15, 24, 15, 14, 12, 15, 14, 14, 10, 14, 13, 15, 14,
       17, 15, 14, 14, 13, 15, 14, 14, 15, 15, 15, 14, 13, 14, 13, 15, 14,
       14, 13, 13, 15, 10, 14, 15, 13, 13, 15, 15, 17, 16, 13, 16, 14, 14,
       16, 13, 13, 15, 15, 17, 14, 15, 15, 14, 14,  4, 16, 15, 15, 14, 15,
       14, 14, 15, 14, 15, 19, 13, 14, 12, 15, 15, 13, 14, 16, 39, 14, 15,
       14, 15, 14, 14, 12, 13, 14, 27]

proton_14406 = [15, 16, 15, 15, 15, 20, 15, 15, 16, 15, 18, 15, 14, 15, 16, 16, 14,
       14, 15, 15, 37, 14, 15, 15, 14, 14]

#
#

ring_22126 = [16, 15, 14, 14, 14, 14, 13, 14, 14, 15, 16, 14, 15, 13, 13, 14, 15,
       29, 15, 13, 14, 23, 13, 14, 15, 15, 16, 14, 13, 15, 15, 14, 13, 13,
       23, 38, 15, 15, 15, 14, 11, 14, 13, 13, 13, 14, 13, 15, 15, 14, 13,
       15, 15, 29, 15, 15, 13, 13, 14, 13, 14, 15, 12, 15, 14, 15, 15, 15,
       14, 14, 10, 13, 15, 15, 15, 14, 14, 34, 39, 14]

proton_22126 = [23, 20, 25, 24, 19, 20, 24,  5, 20, 21, 21, 24, 22, 22, 21, 22, 20,
       23, 20, 22, 21, 23, 24, 24, 22, 23, 20, 19, 21, 21, 22, 20, 23, 23,
       21, 22, 21, 21, 21, 23, 22, 21, 21, 22, 23, 17, 15,  4, 21, 20, 20,
       20, 21, 20, 20, 20, 20, 25, 20, 22, 20, 21, 21, 21, 21, 21, 23]


ring_22365 = [15, 12, 14, 12, 13, 16, 14, 14, 14, 14, 14, 14, 13, 14, 14, 14, 14,
       13, 13, 14, 13, 13, 14, 13, 14, 14, 14, 13, 14, 13, 14, 12, 14, 15,
       24, 14, 15, 13, 13, 14, 13, 13, 19, 13, 14, 13, 15, 13, 14, 13, 12,
       14, 15, 14, 14, 12, 14, 14, 13, 38, 15, 14, 14, 13, 13, 14, 13, 13,
       11, 13, 12, 12, 14, 13, 14, 28, 13, 14, 35, 14, 13, 15, 13, 12, 15,
       28, 15, 12, 11, 12, 13,  8, 13, 13]

proton_22365 = [22, 20, 22, 19, 19, 23, 20, 21, 18, 19, 21, 22, 24, 22, 21, 22, 20,
       19, 14, 24, 24, 21, 22, 20, 21, 20, 22, 20, 23, 21, 22, 20, 23, 21,
       20, 22, 21, 11, 20, 19, 20, 19, 38, 20, 19, 20, 21, 22, 21, 19, 19,
       19, 20, 13, 20, 20, 21, 21, 21, 20, 23, 21, 22]

ring_32298 = [6, 25, 14, 12, 14, 14, 15, 14, 15, 15,  9, 17, 23,  1, 13, 11, 15,
       12, 14, 14, 16, 39, 13, 15, 12, 13,  0, 15, 13, 13, 15, 13,  3, 16,
       11, 13, 14, 15, 29, 14, 14, 25, 14,  5, 14, 11, 14, 19, 14, 14, 28,
       14, 14, 14, 16, 14, 14, 14, 13, 14, 13]

proton_32298 = [11, 19, 19, 14, 13, 15, 14, 24, 37, 15, 13, 17, 18, 16, 17, 18, 17,
       15, 17, 17, 17,  1, 19, 10,  7, 16,  7, 14, 16, 22, 13, 14]

ring_35219 = [29, 15, 14, 11, 13, 13, 16, 13, 13, 14, 13, 13, 14, 12, 13, 13, 15,
       12, 12, 14, 14, 16, 14, 14, 14, 14, 14, 14, 33, 14, 14, 15, 13, 17,
       14, 14,  3, 15, 14, 15, 35, 14, 14, 14, 13, 13, 14, 15, 13, 13, 14,
       14, 14, 19, 31, 12, 13, 14, 13, 15, 13, 15, 14, 14]

proton_35219 = [17, 16, 17, 17, 17, 15, 15, 18, 18, 13, 15, 16, 13, 15, 15]

ring_28000 = [15, 15, 15, 14, 15, 15,  8,  2, 15, 13, 11, 15, 39, 15, 12, 14, 15,
       15, 12, 15, 14, 14, 14, 14, 13, 15, 15, 14, 15, 14, 15, 14, 14, 14,
       15, 11, 14, 14, 15, 14, 14, 17, 17, 14, 13, 14, 15, 14, 16, 13, 12,
       14, 15, 14, 12, 13, 15, 15, 14, 15, 15, 17, 15, 14, 13, 21, 13, 16,
       14, 14, 15, 14, 13, 13, 13, 14, 13, 15, 14, 13,  4, 14, 12, 14, 16,
       14, 14, 14, 10, 15, 24, 15, 14, 13,  5, 14, 13, 12, 16, 14, 11, 13,
       24]

proton_28000 = [19, 24, 21, 20, 21, 20, 22, 21, 22, 21,  4, 22, 21, 19, 19, 18, 21,
       22, 18, 16,  8, 24, 23, 21, 20, 20, 18, 20, 23, 21, 21, 19, 21,  2,
       22, 19, 21, 20]

ring_35298 = [13, 14, 14, 15, 14, 12, 15, 14, 13, 34, 14, 14, 14, 12, 14, 35, 14,
       14, 14, 13, 13, 25, 15, 13, 15, 14,  0, 13, 14, 13, 15, 12, 15, 13,
       13, 13, 13, 12, 13, 14, 13, 13, 14, 14, 15, 13,  3, 14, 13, 14, 13,
       15]

proton_35298 = [14, 14,  5, 36, 15, 19, 18, 15, 10, 21, 21, 18, 19, 19, 20, 19, 19,
       18, 19, 22, 31, 20, 24]


###
###
### Fitted Events

proton_8234 = [ 0, 19, 20,  5, 20, 21, 21, 21, 19, 21, 20, 19, 21, 19, 19, 19, 20,
                20, 16, 21, 18, 21, 22, 18, 18, 19, 20, 20,  0, 20, 19, 20, 18, 20]

ring_8234 = [13, 14, 12, 17, 15, 13, 12, 14, 14, 16, 15, 13, 14, 15, 13, 14, 14,
        0, 13, 15, 13, 15, 13, 14, 14, 15, 14, 13, 15, 13, 16, 15, 14, 15,
       12,  9, 14, 15, 14, 15, 13, 14, 13, 15, 14, 15, 14, 14, 14, 14, 11,
       14,  2, 14, 15, 15, 16, 13, 14, 15, 12]


proton_8448 = [24, 21, 19, 18, 20, 22, 22, 22, 21, 22, 22, 22, 19, 20, 20, 19, 20,
       19, 19, 19, 20, 20, 19]

ring_8448 = [14, 15, 15,  8, 11, 13, 14, 24, 15, 14, 12, 15, 13, 17, 14, 11, 13,
       15, 20, 35, 16, 14, 17, 17, 14, 13, 15, 15, 14, 13, 13, 15, 13, 14,
       15, 15, 13, 16, 14, 13, 14, 15, 12, 15, 14, 15, 11, 17, 14, 13, 13,
       13, 16, 15, 14, 14, 14, 14, 15]

proton_9404 = [22, 22, 19, 22, 20, 21, 20, 21, 22, 20,  3, 19, 21, 23, 20, 19, 23,
       21, 19, 21, 23, 20, 18, 21, 23]

ring_9404 = [12, 14, 13, 12, 13, 13, 14, 24, 14, 12, 14, 13,  4, 13, 13, 12, 14,
       14, 21, 13, 13, 14, 13, 13, 14, 18, 14, 13, 15, 16, 13, 14, 18, 14,
       35, 14, 12, 13, 14, 15, 13, 23, 15, 14, 31, 14, 13, 14, 14, 13, 24,
       14, 15, 14, 13, 13, 14, 13,  9, 12, 11,  9, 13, 14, 13, 30, 13, 15,
       14, 12, 14, 12, 15, 15, 13, 22, 16, 15, 22, 12, 14, 13, 14, 13, 14,
       13, 15, 14, 17, 14, 15, 15, 13, 13, 12, 12, 15, 29, 14, 13, 16, 15,
       15, 15, 13]

ring_9433 = [31, 13, 14, 15,  8, 12, 14, 14, 13, 13, 14, 14, 13, 14, 15, 32, 14,
       14, 13, 15, 12, 15, 15, 12, 14, 14, 14, 14, 17, 13, 14, 14, 14, 13,
       14,  7, 14, 15, 12, 14, 14, 15, 13, 14, 14, 14, 33, 15, 13, 14, 12,
       15, 13, 13, 15, 13, 14, 14, 13, 14, 16, 14, 12, 15, 23, 12, 17, 14,
       13, 23, 15]

proton_9433 = [33, 21, 19, 20, 19, 19, 19, 18, 20, 20, 21, 20, 11, 20, 21, 21, 18,
       19, 20]


ring_27374 = [12, 15, 24, 14, 15, 13,  8, 13, 13, 15, 13, 39, 11, 14, 30, 14, 14,
       12, 14, 21, 14, 14, 13, 14, 14, 16, 13, 12, 13, 14, 15, 15, 15, 14,
       24, 15, 13, 16, 14, 14, 14, 14, 15,  0, 13, 14, 14, 14, 13, 14, 15,
       14, 14, 14, 12, 15, 14, 12, 12, 15, 14, 14, 13, 15, 14, 15, 15, 11,
       14, 14, 14, 14, 14, 13, 12, 13,  1, 14, 23, 13, 14, 36, 14, 22, 13,
       14, 15, 11, 25, 14, 16, 26,  9, 14, 15, 13, 13, 14, 13, 12, 14, 13,
       14]
proton_27374 = [18, 16, 17, 17, 17, 17, 31, 19, 16,  1, 13]

ring_28328 = [13, 14, 14, 13, 16, 14, 17, 15, 13, 13, 14, 15, 14, 15, 15, 14, 16,
       16, 14, 14, 12, 13, 15, 13, 11, 23, 14, 15, 16, 33, 13, 14, 14, 15,
       13, 13, 15, 13,  1, 15, 15, 14, 15, 14, 22, 15, 13, 13, 15, 15, 15,
       15, 15, 16,  7, 27, 12, 16, 14, 22, 13, 15, 13, 13, 14, 12,  1, 15,
       15,  0, 14, 15, 13, 16, 13, 14, 14,  7, 15, 36, 14, 14, 13, 15, 27,
       14, 13, 14, 15, 13, 14,  2, 14, 12, 15, 26, 19, 10, 13, 15, 13, 12,
       26, 14,  0, 22]
proton_28328 = [ 5, 36, 19, 18, 10, 21, 21, 18, 19, 19, 20, 19, 19, 18, 19, 22, 31,
       20, 24]
###
###
###


### Useful functions from lstchain analysis

In [ ]:
def read_calibration_file(path):
    """
    Read the correction from hdf5 calibration file
    """

    with tables.open_file(path) as f:
        tel_groups = [
            key for key in f.root._v_children.keys()
            if key.startswith('tel_')
        ]

    mon = MonitoringContainer()

    for base in tel_groups:
        with HDF5TableReader(path) as h5_table:
            # read the calibration data
            tel_id = int(base[4:])
            mon.tel[tel_id] = MonitoringCameraContainer(
                calibration=next(h5_table.read(f'/{base}/calibration', WaveformCalibrationContainer)),
                pedestal=next(h5_table.read(f'/{base}/pedestal', PedestalContainer)),
                flatfield=next(h5_table.read(f'/{base}/flatfield', FlatFieldContainer)),
                pixel_status=next(h5_table.read(f"/{base}/pixel_status", PixelStatusContainer)),
            )
    return mon

## Uploading all data

In [ ]:
filename = '/Users/vdk/RealLST/typical_run_data/LST-1.1.Run14948.0001.fits.fz'
source = EventSource(filename)
camgeom = source.subarray.tel[1].camera.geometry
file = open('/Users/vdk/RealLST/flag_cut.pkl', 'rb')
#file = open('/Users/vdk/RealLST/flag_cut_insideTAGPIXTHR.pkl', 'rb')
#file = open('/Users/vdk/RealLST/flag_cut_quaterOfRun.pkl', 'rb')
data = pickle.load(file)
muon_file = '/Users/vdk/RealLST/muons_LST-1.Run14948.0001.fits' # Full 4 streams lstchain output
hdul = fits.open(muon_file)
hdul.info()
opt_eff = []
event_id = []
# element number 10 is muon efficiency, so in such way we can check it
for i in range(len(hdul[1].data)):
    opt_eff.append(hdul[1].data[i][10])
    event_id.append(hdul[1].data[i][0])
print(len(data))
print(data.keys())

In [ ]:
muon_dict = {}
for key in data:
    if key in event_id:
        muon_dict[key] = copy.deepcopy(data[key])

print(muon_dict.keys())

### Images of fitted muons

In [ ]:
for key in muon_dict:
    title = f'event_id = {key}'
    disp = CameraDisplay(camgeom,title=title)
    #disp.image = event.r1.tel[teln].waveform[:,11]
    #disp.image = event.dl0.tel[teln].waveform[:,11]
    disp.image = data[key].dl1.tel[1].image
    disp.cmap = plt.cm.RdBu_r
    disp.add_colorbar()
    disp.set_limits_percent(95)
    plt.show()

plt.close()


### Images of not muons

In [ ]:
check = 0
for key in data:
    if key in all_protons:
        check+=1
        title = f'event_id = {key}'
        disp = CameraDisplay(camgeom,title=title)
        #disp.image = event.r1.tel[teln].waveform[:,11]
        #disp.image = event.dl0.tel[teln].waveform[:,11]
        disp.image = data[key].dl1.tel[1].image
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        plt.show()

plt.close()

In [ ]:
check

In [ ]:
arg_max_muons = []
arg_max_protons = []
arg_max_muons_mask = []
arg_max_protons_mask = []
muons = 0
protons = 0

for key in muon_dict:
    pixel_mask = np.array(np.max(data[key].r1.tel[1].waveform, axis = 1) > 2.5, dtype = bool)
    for pixel in data[key].r1.tel[1].waveform[pixel_mask]:
        arg_max_muons_mask.append(np.argmax(pixel))

for key in data:
    pixel_mask = np.array(np.max(data[key].r1.tel[1].waveform, axis = 1) > 2.5, dtype = bool)
    if key in strict_protons:
        for pixel in data[key].r1.tel[1].waveform[pixel_mask]:
            arg_max_protons_mask.append(np.argmax(pixel))

In [ ]:
k, z, l = plt.hist(arg_max_protons_mask, bins = 36, range = (0,36), density = True, alpha = 0.35, label = 'protons')
k, z, l = plt.hist(arg_max_muons_mask, bins = 36, range = (0,36), density = True, alpha = 0.35, label = 'muons')
plt.legend()
plt.xlabel("Time of max R1 amplitude")
plt.ylabel("Probability density")
plt.grid()
plt.ylim(0.01, 0.35)
plt.grid(alpha = 0.5)

In [ ]:
ring_stack = np.concatenate((ring_2223, ring_9222,ring_2438, ring_25051, ring_14406, ring_22126, ring_22365, ring_32298, ring_35219, ring_8234, ring_8448, ring_9404, ring_9433, ring_27374, ring_28328, ring_28000, ring_35298), axis=None)
proton_stack = np.concatenate((proton_2223, proton_9222,proton_2438, proton_25051, proton_14406, proton_22126, proton_22365, proton_32298, proton_35219, proton_8234, proton_8448, proton_9404, proton_9433, proton_27374, proton_28328, proton_28000, proton_35298), axis=None)
plt.figure(figsize=(12,9))
plt.hist(proton_stack, density = True, alpha = 0.45, bins = 39,label = 'protons')
plt.hist(ring_stack, density = True, alpha = 0.45,bins =39,label = 'muons')
#plt.title("Distribution of pixel waveform peak times for contaminated muon rings (17 Events)")
plt.xlabel("Time of max value R1 waveform")
plt.ylabel("Probability Density")
plt.legend()
plt.grid(alpha = 0.5)
plt.ylim(0.01,0.35)
#plt.savefig("/Users/vdk/ImagePurgatorium/TimeSamplesHistogram.png")
print(len(ring_stack), len(proton_stack))

## Cleaning

In [ ]:
file = open('/Users/vdk/RealLST/flag_cut.pkl', 'rb')
data = pickle.load(file)
print("all keys - ", data.keys())
print("muon rings - ", event_id)

###  **Cleaning prosedure:** 
<h5>

1.  Create the copy for the original R1 waveform to occasionally not change it. 

2. Iterate through every pixel in the event, and find the time of the R1 waveform maximum in this pixel.

3. Create a boolean mask with pixels in which the time of R1 waveform maximum lies in the required cleaning range - let's call them "cleaned pixels".

4. Assign the mean value of the R1 waveform to the "cleaned pixels".

5. Count the number of the islands among "cleaned pixels".

6. Return the original R1 waveforms value to the "cleaned pixels" which form islands with a number of pixels <=3

7. Voila - now we have an event, in which R1 waveforms that should belong to the muon rings are non-changed, but waveforms that can be contaminated by showers have some mean value that is close to zero

<h5>

### Images to check what pixels are cleaned

In [ ]:
event = copy.deepcopy(data[62395]) # True muon
#event = copy.deepcopy(data[87150]) # True proton
#event = copy.deepcopy(data[72806]) # muon/proton

pixel_mask = []
island_clean_mask = []
tel_id = 1

original_r1_waveform = event.r1.tel[tel_id].waveform.copy()
image_wave = copy.deepcopy(event.r1.tel[tel_id].waveform)


#for pixel_number, pixel in enumerate(event.r0.tel[tel_id].waveform[1]):
for pixel_number, pixel in enumerate(event.r1.tel[tel_id].waveform):
    if np.argmax(pixel) in range(15,36): 
        pixel_mask.append(True)
    else:
        pixel_mask.append(False)


working_wave = event.r1.tel[tel_id].waveform.copy()

for pixel_number, pixel in enumerate(working_wave):
    if pixel_mask[pixel_number]:
        island_clean_mask.append(True)
        #working_wave[pixel_number] = np.zeros(36)
        #working_wave[pixel_number] = np.ones(36)     
        working_wave[pixel_number] = np.mean(event.r1.tel[tel_id].waveform, axis = 0)
    else:
        island_clean_mask.append(False)


event.r1.tel[tel_id].waveform = working_wave

fig, axes = plt.subplots(2, 1, sharex=False, gridspec_kw={'height_ratios': [1, 1]}, figsize=(13,13))
fig.subplots_adjust(hspace=0.2)
ax = axes[0]
camgeom = source.subarray.tel[1].camera.geometry
#title=f"event{iter_number}_fit{round(j.muon.tel[teln].efficiency.optical_efficiency,3)}_width{round(j.muon.tel[teln].efficiency.width.to_value(),3)}"
title1 = f'event{key} before cleaning'
disp = CameraDisplay(camgeom,title=title1, ax = ax)
disp.image = image_wave.sum(axis = 1)
disp.cmap = plt.cm.RdBu_r
#disp.highlight_pixels(pixel_mask, color = 'green', alpha = 1, linewidth = 2)
disp.add_colorbar()
#disp.set_limits_percent(95)
disp.set_limits_minmax(0,50)
bx = axes[1]
title2 = f'event{key} after cleaning with highlighted pixels cleaned based on time of arrival'
disp = CameraDisplay(camgeom,title=title2, ax = bx)
disp.image = event.r1.tel[tel_id].waveform.sum(axis = 1)
disp.highlight_pixels(pixel_mask, color = 'green', alpha = 1, linewidth = 1.5)
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
#disp.set_limits_percent(75)
disp.set_limits_minmax(0,50)
plt.show()

for_island_image = copy.deepcopy(event.r1.tel[tel_id].waveform.sum(axis = 1))

# calib(event)
# image_processor(event)
# cam_geometry = source.subarray.tel[tel_id].camera.geometry
# return_pixels = np.array([])
# island_clean_mask = np.array(for_clean_mask)
# n_islands, island_id = number_of_islands(cam_geometry, for_clean_mask)

# for t in range(1,n_islands+1):
#     if len(np.where(island_id == t)[0]) <= 3:
#         return_pixels = np.append(return_pixels,np.where(island_id ==t)[0])

# return_pixels = return_pixels.astype(int)

# event.r1.tel[tel_id].waveform[return_pixels,:] = original_r1_waveform[return_pixels,:] 

In [ ]:
#print(island_clean_mask)
#print(pixel_mask)

# for i,j in enumerate(pixel_mask):
#     if not j == island_clean_mask[i]:
#         print(f"pixel_mask = {j}, island_mask = {island_clean_mask[i]}")
np.max(np.max(image_wave, axis = 1))

In [ ]:
np.mean(event.r1.tel[tel_id].waveform, axis = 0)

### Images with island cleaning

In [ ]:
event = copy.deepcopy(data[62395]) # True muon
#event = copy.deepcopy(data[87150]) # True proton
#event = copy.deepcopy(data[72806]) # muon/proton

pixel_mask = []
island_clean_mask = []
tel_id = 1

original_r1_waveform = event.r1.tel[tel_id].waveform.copy()
image_wave = copy.deepcopy(event.r1.tel[tel_id].waveform)


#for pixel_number, pixel in enumerate(event.r0.tel[tel_id].waveform[1]):
for pixel_number, pixel in enumerate(event.r1.tel[tel_id].waveform):
    if np.argmax(pixel) in range(15,36): 
        pixel_mask.append(True)
    else:
        pixel_mask.append(False)


working_wave = event.r1.tel[tel_id].waveform.copy()

for pixel_number, pixel in enumerate(working_wave):
    if pixel_mask[pixel_number]:
        island_clean_mask.append(True)
        #working_wave[pixel_number] = np.zeros(36)
        #working_wave[pixel_number] = np.ones(36)     
        working_wave[pixel_number] = np.mean(event.r1.tel[tel_id].waveform, axis = 0)
    else:
        island_clean_mask.append(False)


event.r1.tel[tel_id].waveform = working_wave


cam_geometry = source.subarray.tel[tel_id].camera.geometry
return_pixels = np.array([])
island_clean_mask = np.array(island_clean_mask)

n_islands, island_id = number_of_islands(cam_geometry, island_clean_mask)

for island in range(1,n_islands+1):
    if len(np.where(island_id == island)[0]) <= 3:
        return_pixels = np.append(return_pixels,np.where(island_id ==island)[0])

return_pixels = return_pixels.astype(int)
print(return_pixels)
event.r1.tel[tel_id].waveform[return_pixels,:] = original_r1_waveform[return_pixels,:] 

fig, axes = plt.subplots(2, 1, sharex=False, gridspec_kw={'height_ratios': [1, 1]}, figsize=(13,13))
fig.subplots_adjust(hspace=0.2)
ax = axes[0]
camgeom = source.subarray.tel[1].camera.geometry
#title=f"event{iter_number}_fit{round(j.muon.tel[teln].efficiency.optical_efficiency,3)}_width{round(j.muon.tel[teln].efficiency.width.to_value(),3)}"
title1 = f'event{key} after cleaning'
disp = CameraDisplay(camgeom,title=title1, ax = ax)
#disp.image = image_wave.sum(axis = 1)
disp.image = for_island_image
disp.cmap = plt.cm.RdBu_r
#disp.highlight_pixels(pixel_mask, color = 'green', alpha = 1, linewidth = 0.65)
disp.add_colorbar()
#disp.set_limits_percent(95)
disp.set_limits_minmax(0,50)
bx = axes[1]
title2 = f'event{key} after cleaning with highlighted pixels which are returned by island cleaning'
disp = CameraDisplay(camgeom,title=title2, ax = bx)
disp.image = event.r1.tel[tel_id].waveform.sum(axis = 1)
disp.highlight_pixels(return_pixels, color = 'green', alpha = 1, linewidth = 3.5)
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
#disp.set_limits_percent(95)
disp.set_limits_minmax(0,50)
plt.show()

In [ ]:
np.mean(event.r1.tel[tel_id].waveform, axis = 0).shape

### Functional realisation of cleaning for monoscopic case

In [ ]:
def shower_cleaning(event, subarray):
    pixel_mask = []
    island_clean_mask = []
    start_time = 15
    stop_time = 36
    tel_id = 1

    original_r1_waveform = event.r1.tel[tel_id].waveform.copy()

    for pixel_number, pixel in enumerate(event.r1.tel[tel_id].waveform):
        if np.argmax(pixel) in range(start_time,stop_time): 
            pixel_mask.append(True)
        else:
            pixel_mask.append(False)


    working_wave = event.r1.tel[tel_id].waveform.copy()

    for pixel_number, pixel in enumerate(working_wave):
        if pixel_mask[pixel_number]:
            island_clean_mask.append(True)
            #working_wave[pixel_number] = np.zeros(36)
            working_wave[pixel_number] = np.mean(event.r1.tel[tel_id].waveform, axis = 0)
        else:
            island_clean_mask.append(False)


    event.r1.tel[tel_id].waveform = working_wave


    cam_geometry = subarray.tel[tel_id].camera.geometry
    return_pixels = np.array([])
    island_clean_mask = np.array(island_clean_mask)
    n_islands, island_id = number_of_islands(cam_geometry, island_clean_mask)

    for island in range(1,n_islands+1):
        if len(np.where(island_id == island)[0]) <= 3:
            return_pixels = np.append(return_pixels,np.where(island_id ==island)[0])

    return_pixels = return_pixels.astype(int)
    event.r1.tel[tel_id].waveform[return_pixels,:] = original_r1_waveform[return_pixels,:]
    return event.r1.tel[tel_id].waveform

### Implement cleaning with lstchain

In [ ]:
muon_dict = {}
for key in data:
    if key in event_id:
        muon_dict[key] = copy.deepcopy(data[key])

print(muon_dict.keys())

In [ ]:
config_filename = '/Users/vdk/RealLST/typical_run_data/lstchain_standard_v0.10_heuristic_ff.json'
with open(config_filename) as json_file:
    config_data = json.load(json_file)

custom_config = Config(config_data)


cfg = Config({
    "source_config": {
        "EventSource" : {
            "allowed_tels": [1],"max_events": 9999}},
        "PointingSource":{
            "drive_report_path": '/Users/vdk/RealLST/data/DrivePosition_log_20231007.txt'},
        "LSTR0Corrections": {
          "calib_scale_high_gain":1.088,
          "calib_scale_low_gain":1.004,
          "drs4_pedestal_path": '/Users/vdk/RealLST/data/drs4_pedestal.Run14937.0000.h5',
          "calibration_path": '/Users/vdk/RealLST/data/calibration_filters_52.Run14938.0000.h5',
          "drs4_time_calibration_path": '/Users/vdk/RealLST/data/time_calibration.Run08349.0000.h5'
      }})

config = replace_config(standard_config, custom_config)

filename = '/Users/vdk/RealLST/typical_run_data/LST-1.1.Run14948.0001.fits.fz'
source = EventSource(filename, config = Config(cfg))


In [ ]:
image_processor = ImageProcessor(source.subarray)
muon_processor = MuonProcessor(source.subarray)
calib = CameraCalibrator(image_extractor_type=config['image_extractor_for_muons'],subarray = source.subarray)

In [ ]:
opt_eff = []
for key in muon_dict:
    event = muon_dict[key]
    bad_pixels = calibration_mon.unusable_pixels[0]

    # Set to 0 unreliable pixels:
    image = event.dl1.tel[1].image*(~bad_pixels)
    calibration_mon = source.r0_r1_calibrator.mon_data.tel[tel_id].calibration
    numsamples = event.r1.tel[tel_id].waveform.shape[1]  # not necessarily the same as in r0!
    bad_pixels_hg = calibration_mon.unusable_pixels[0]
    bad_pixels_lg = calibration_mon.unusable_pixels[1]

    # # Now set to 0 all samples in unreliable pixels. Important for global peak
    # # integrator in case of crazy pixels!  TBD: can this be done in a simpler
    # # way?
    bad_pixels = bad_pixels_hg | bad_pixels_lg
    bad_waveform = np.transpose(np.array(numsamples*[bad_pixels]))

    # # print('hg bad pixels:',np.where(bad_pixels_hg))
    # # print('lg bad pixels:',np.where(bad_pixels_lg))

    calib(event)
    event.r1.tel[tel_id].waveform *= ~bad_waveform
    image_processor(event)
    muonintensityparam, dist_mask, \
    ring_size, size_outside_ring, muonringparam, \
    good_ring, radial_distribution, \
    mean_pixel_charge_around_ring,\
    muonpars = \
        analyze_muon_event(source.subarray,
                            tel_id, event.index.event_id,
                            event.dl1.tel[1].image, good_ring_config=None,
                            plot_rings=False, plots_path='')
    
    opt_eff.append(muonintensityparam.optical_efficiency)


In [ ]:
opt_eff = np.array(opt_eff)
opt_eff = opt_eff[np.logical_not(np.isnan(opt_eff))]

print(len(opt_eff))
np.mean(opt_eff)

In [ ]:
calib(event)

In [ ]:
print(event.dl1.tel[1])

In [ ]:
muonintensityparam, dist_mask, \
ring_size, size_outside_ring, muonringparam, \
good_ring, radial_distribution, \
mean_pixel_charge_around_ring,\
muonpars = \
    analyze_muon_event(source.subarray,
                        tel_id, event.index.event_id,
                        event.dl1.tel[1].image, good_ring_config=None,
                        plot_rings=False, plots_path='')

In [ ]:
muonintensityparam.optical_efficiency

In [ ]:
opt_eff = []
for key in muon_dict:
    event = muon_dict[key]
    calib(event)
    muonintensityparam, dist_mask, \
    ring_size, size_outside_ring, muonringparam, \
    good_ring, radial_distribution, \
    mean_pixel_charge_around_ring,\
    muonpars = \
        analyze_muon_event(source.subarray,
                            tel_id, event.index.event_id,
                            event.dl1.tel[1].image, good_ring_config=None,
                            plot_rings=False, plots_path='')
    opt_eff.append(muonintensityparam.optical_efficiency)


In [ ]:
opt_eff = np.array(opt_eff)
opt_eff = opt_eff[np.logical_not(np.isnan(opt_eff))]

np.mean(opt_eff)

In [ ]:
#for telescope_id, dl1_tel in event.dl1.tel.items():
#    print(dl1_tel)

print(event.dl1.tel[1])

### Check lstchain process


In [ ]:
event = copy.deepcopy(muon_dict[62395])

In [ ]:
event.pointing.tel[1].array_azimuth

In [ ]:
event = copy.deepcopy(muon_dict[62395])
telescope_id = 1
dl1_tel = event.dl1.tel[1]
min_pe_for_muon_t_calc = 10.

subarray = source.subarray

mon_data = read_calibration_file('/Users/vdk/RealLST/data/calibration_filters_52.Run14938.0000.h5')

calibration_mon = mon_data.tel[telescope_id].calibration

bad_pixels = calibration_mon.unusable_pixels[0]
image = dl1_tel.image*(~bad_pixels)

# re-calibrate r1 to obtain new dl1, using a more adequate pulse integrator for muon rings
numsamples = event.r1.tel[telescope_id].waveform.shape[1]  # not necessarily the same as in r0!
bad_pixels_hg = calibration_mon.unusable_pixels[0]
bad_pixels_lg = calibration_mon.unusable_pixels[1]

# Now set to 0 all samples in unreliable pixels. Important for global peak
# integrator in case of crazy pixels!  TBD: can this be done in a simpler
# way?
bad_pixels = bad_pixels_hg | bad_pixels_lg
bad_waveform = np.transpose(np.array(numsamples*[bad_pixels]))

# print('hg bad pixels:',np.where(bad_pixels_hg))
# print('lg bad pixels:',np.where(bad_pixels_lg))

event.r1.tel[telescope_id].waveform *= ~bad_waveform
calib(event)

# since ctapipe 0.17,  the calibrator overwrites the full dl1 container
# instead of overwriting the image in the existing container
# so we need to get the image again

image = event.dl1.tel[telescope_id].image * (~bad_pixels)

# Check again: with the extractor for muon rings (most likely GlobalPeakWindowSum)
# perhaps the event is no longer promising (e.g. if it has a large time evolution)
if not tag_pix_thr(image):
    good_ring = False
else:
    muonintensityparam, dist_mask, \
    ring_size, size_outside_ring, muonringparam, \
    good_ring, radial_distribution, \
    mean_pixel_charge_around_ring,\
    muonpars = \
        analyze_muon_event(subarray,
                        tel_id, event.index.event_id,
                        image, good_ring_config=None,
                        plot_rings=False, plots_path='')
    #                      plot_rings=True, plots_path='./')
    #           (test) plot muon rings as png files


In [ ]:
MUON INTENSITY PARAM of event62395 =  {'impact': <Quantity 3.81654634 m>,
 'impact_x': <Quantity 3.74681163 m>,
 'impact_y': <Quantity -0.7262428 m>,
 'optical_efficiency': 0.1591262595259284,
 'width': <Quantity 0.06481282 deg>}
print(muonintensityparam)

In [ ]:
camgeom = source.subarray.tel[1].camera.geometry
#title=f"event{iter_number}_fit{round(j.muon.tel[teln].efficiency.optical_efficiency,3)}_width{round(j.muon.tel[teln].efficiency.width.to_value(),3)}"
title1 = f'event{event.index.event_id} before cleaning with highlighted pixels cleaned based on time of arrival'
disp = CameraDisplay(camgeom,title=title1)
disp.image = event.dl1.tel[1].image
disp.cmap = plt.cm.RdBu_r
#disp.highlight_pixels(pixel_mask, color = 'black', alpha = 0.9, linewidth = 0.65)
disp.add_colorbar()
disp.set_limits_percent(95)


## Try not with pickle but small sample

In [ ]:
filename = '/Users/vdk/RealLST/typical_run_data/LST-1.1.Run14948.0001.fits.fz'
camgeom = source.subarray.tel[1].camera.geometry

config_filename = '/Users/vdk/RealLST/typical_run_data/lstchain_standard_v0.10_heuristic_ff.json'
with open(config_filename) as json_file:
    config_data = json.load(json_file)

custom_config = Config(config_data)


cfg = Config({
    "source_config": {
        "EventSource" : {
            "allowed_tels": [1],"max_events": 9999}},
        "PointingSource":{
            "drive_report_path": '/Users/vdk/RealLST/data/DrivePosition_log_20231007.txt'},
        "LSTR0Corrections": {
          "calib_scale_high_gain":1.088,
          "calib_scale_low_gain":1.004,
          "drs4_pedestal_path": '/Users/vdk/RealLST/data/drs4_pedestal.Run14937.0000.h5',
          "calibration_path": '/Users/vdk/RealLST/data/calibration_filters_52.Run14938.0000.h5',
          "drs4_time_calibration_path": '/Users/vdk/RealLST/data/time_calibration.Run08349.0000.h5'
      }})

config = replace_config(standard_config, custom_config)

source = EventSource(filename, config = Config(cfg))


In [ ]:
ctapipe.__version__